In [1]:
from tensorflow.keras.models import load_model
import tensorflow.keras.backend as K
import pandas as pd
from sklearn.preprocessing import LabelEncoder

2024-03-25 12:55:14.347741: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Define custom F1 metric and its dependencies
def f1(y_true, y_pred):
    def recall_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = TP / (Positives + K.epsilon())
        return recall

    def precision_m(y_true, y_pred):
        TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        Pred_Positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = TP / (Pred_Positives + K.epsilon())
        return precision

    precision, recall = precision_m(y_true, y_pred), recall_m(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))
    
# Load all trained models
model_paths = [
    "Model_lstm1000000_sentiment_V4",
    "Model_lstm1000000_V7_3",
    "trained_cardiff10",
    "Model_lstm1000000_V6_2",
    "Model_lstm1000000_sentiment_V3"
    # Make sure paths are correct
]

# Encode labels
le = LabelEncoder()

# Corresponding weights for each model, with higher weights for more accurate models
model_weights = [2.0, 1.0, 2.0, 1.0, 1.0]  # Example weights
# Load all models, including the custom F1 metric
models = [load_model(path, custom_objects={'f1': f1}) for path in model_paths]

# Prepare the test set (your existing code handles this)

# Predict with each model and average predictions
predictions = [model.predict(test_padded, batch_size=128) for model in models]
avg_predictions = np.mean(predictions, axis=0)

# Convert averaged predictions to predicted labels
predicted_labels = np.argmax(avg_predictions, axis=1)
predicted_emotions = le.inverse_transform(predicted_labels)

# Prepare the submission DataFrame
submission_df = pd.DataFrame({
    'id': test_df.index,  # Use original test DataFrame index as ID
    'emotion': predicted_emotions
})

# Save the submission file
submission_df.to_csv('ensemble_v1.csv', index=False, sep=',')

ValueError: File format not supported: filepath=Model_lstm1000000_sentiment_V4. Keras 3 only supports V3 `.keras` files and legacy H5 format files (`.h5` extension). Note that the legacy SavedModel format is not supported by `load_model()` in Keras 3. In order to reload a TensorFlow SavedModel as an inference-only layer in Keras 3, use `keras.layers.TFSMLayer(Model_lstm1000000_sentiment_V4, call_endpoint='serving_default')` (note that your `call_endpoint` might have a different name).